In [4]:
import numpy as np
import SimpleITK as sitk

from itkwidgets import view
from scipy.ndimage import zoom
from scipy.io import savemat, loadmat
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm as tqdm
sys.path.append('Packages')
from data import convert

%matplotlib widget

## Gaussian random field generation

In [5]:
def fftIndgen(n):
    a = list(range(0, int(n/2+1)))
    b = list(range(1, int(n/2)))
    b = b[::-1]
    b = [-i for i in b]
    return a + b

def gaussian_random_field(Pk = lambda k : k**-3.0, size = 100):
    def Pk2(kx, ky):
        if kx == 0 and ky == 0:
            return 0.0
        return np.sqrt(Pk(np.sqrt(kx**2 + ky**2)))
    noise = np.fft.fft2(np.random.normal(size = (size, size)))
    amplitude = np.zeros((size,size))
    for i, kx in enumerate(fftIndgen(size)):
        for j, ky in enumerate(fftIndgen(size)):            
            amplitude[i, j] = Pk2(kx, ky)
    return np.fft.ifft2(noise * amplitude)

In [8]:
size, sample_num = 128, 200
grf_batch = np.zeros((sample_num,size,size))
for i in tqdm(range(sample_num)):
    alpha = -2.3
    out = gaussian_random_field(Pk = lambda k: k**alpha, size=size)
    grf_batch[i] = out.real

savemat(f'grf/grf_{abs(alpha)}_size{size}_num{sample_num}.mat', {"grf": grf_batch})

  0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
plt.figure()
plt.imshow(grf_batch[0], interpolation='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Breast phantom generation

In [3]:
# path = '/home/sci/hdai/Projects/Dataset/Ultrasound/Phantom/Neg_07_Left'

# save_name = f'{path}/PreData_Interp.nhdr'
# predata_interp = loadmat(f'{path}/PreData_Interp.mat')['preData']
# sitk.WriteImage(sitk.GetImageFromArray(predata_interp), save_name)

# save_name = f'{path}/PostData_Interp.nhdr'
# postdata_interp = loadmat(f'{path}/PostData_Interp.mat')['postData']
# sitk.WriteImage(sitk.GetImageFromArray(postdata_interp), save_name)

# save_name = f'{path}/MergedPhantom.nhdr'
# merged_phantom = loadmat(f'{path}/MergedPhantom.mat')['phan']
# sitk.WriteImage(sitk.GetImageFromArray(merged_phantom), save_name)

# h, w, d = predata_interp.shape

In [2]:
subject_name = 'Neg_47_Left'
path = f'/home/sci/hdai/Projects/Dataset/Ultrasound/Phantom/{subject_name}'

predata_interp = convert.read_nhdr(f'{path}/PreData_Interp.nhdr', as_type='numpy')
postdata_interp = convert.read_nhdr(f'{path}/PostData_Interp.nhdr', as_type='numpy')
merged_phantom = convert.read_nhdr(f'{path}/MergedPhantom.nhdr', as_type='numpy')

h, w, d = predata_interp.shape

In [3]:
slice_id = h//2
plt.figure(figsize=(9,3))
plt.subplot(1, 3, 1)
plt.imshow(predata_interp[slice_id])
plt.subplot(1, 3, 2)
plt.imshow(postdata_interp[slice_id])
plt.subplot(1, 3, 3)
plt.imshow(merged_phantom[slice_id])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
desired_length, spared_length = 80, 128
bazel = int((spared_length-desired_length)/2)
predata_interp_scaled = zoom(predata_interp, (desired_length/h, desired_length/w, desired_length/d))
postdata_interp_scaled = zoom(postdata_interp, (desired_length/h, desired_length/w, desired_length/d))
merged_phantom_scaled = zoom(merged_phantom, (desired_length/h, desired_length/w, desired_length/d))

predata_interp_spared = np.zeros((spared_length,spared_length,spared_length))
predata_interp_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = predata_interp_scaled
postdata_interp_spared = np.zeros((spared_length,spared_length,spared_length))
postdata_interp_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = postdata_interp_scaled
merged_phantom_spared = np.zeros((spared_length,spared_length,spared_length))
merged_phantom_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = merged_phantom_scaled

In [7]:
mask = np.where(merged_phantom_spared==0,0,1)
postdata_interp_final = postdata_interp_spared*mask+(1-mask)*1500
view(postdata_interp_final)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [12]:
plt.figure()
plt.imshow(postdata_interp_final[64], cmap='rainbow')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
average_value = np.zeros(7)
average_value[0] = 1500
for i in range(1,7):
    average_value[i] = np.sum(np.where(merged_phantom_spared==i,1,0)*postdata_interp_spared)/np.sum(np.where(merged_phantom_spared==i,1,0))
    
merged_phantom_final = np.zeros_like(merged_phantom_spared)
for i in range(0,7):
    merged_phantom_final += np.where(merged_phantom_spared==i,1,0)*average_value[i]
    
view(merged_phantom_final)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [13]:
phantom_list, postdata_list = [], []

for subject_name in tqdm(['Neg_07_Left','Neg_35_Left','Neg_47_Left']):
    path = f'/home/sci/hdai/Projects/Dataset/Ultrasound/Phantom/{subject_name}'

    predata_interp = convert.read_nhdr(f'{path}/PreData_Interp.nhdr', as_type='numpy')
    postdata_interp = convert.read_nhdr(f'{path}/PostData_Interp.nhdr', as_type='numpy')
    merged_phantom = convert.read_nhdr(f'{path}/MergedPhantom.nhdr', as_type='numpy')

    h, w, d = predata_interp.shape
    
    desired_length, spared_length = 80, 128
    bazel = int((spared_length-desired_length)/2)
    predata_interp_scaled = zoom(predata_interp, (desired_length/h, desired_length/w, desired_length/d))
    postdata_interp_scaled = zoom(postdata_interp, (desired_length/h, desired_length/w, desired_length/d))
    merged_phantom_scaled = zoom(merged_phantom, (desired_length/h, desired_length/w, desired_length/d))

    predata_interp_spared = np.zeros((spared_length,spared_length,spared_length))
    predata_interp_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = predata_interp_scaled
    postdata_interp_spared = np.zeros((spared_length,spared_length,spared_length))
    postdata_interp_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = postdata_interp_scaled
    merged_phantom_spared = np.zeros((spared_length,spared_length,spared_length))
    merged_phantom_spared[bazel:desired_length+bazel,bazel:desired_length+bazel,bazel:desired_length+bazel] = merged_phantom_scaled
    
    mask = np.where(merged_phantom_spared==0,0,1)
    postdata_interp_final = postdata_interp_spared*mask+(1-mask)*1500

    average_value = np.zeros(7)
    average_value[0] = 1500
    for i in range(1,7):
        average_value[i] = np.sum(np.where(merged_phantom_spared==i,1,0)*postdata_interp_spared)/np.sum(np.where(merged_phantom_spared==i,1,0))

    merged_phantom_final = np.zeros_like(merged_phantom_spared)
    for i in range(0,7):
        merged_phantom_final += np.where(merged_phantom_spared==i,1,0)*average_value[i]
        
    for i in range(40,101,3):
        phantom_list.append(merged_phantom_final[i])
        postdata_list.append(postdata_interp_final[i])
        
phantom = np.stack(phantom_list,axis=0)
postdata = np.stack(postdata_list,axis=0)

  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
savemat(f'/home/sci/hdai/Projects/Dataset/Ultrasound/Phantom/phantom_size128_num63.mat', {"phantom": phantom})
savemat(f'/home/sci/hdai/Projects/Dataset/Ultrasound/Phantom/postdata_size128_num63.mat', {"postdata": postdata})

In [15]:
view(postdata)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…